# Downloading COVID Data From FloridaDisaster.ORG

In [18]:
from selenium import webdriver
import os
import wget
import PyPDF2 as pdf
from PyPDF2 import PdfFileReader, PdfFileWriter
import time
import textract
from pathlib import Path
import re
import pandas as pd

In [2]:
chrome_path = r"chromedriver.exe"

## Initializing driver element and finding 

In [3]:
driver = webdriver.Chrome(chrome_path)
driver.get('https://floridadisaster.org/covid19/covid-19-data-reports/')
title = driver.title
time.sleep(3)
result = driver.find_elements_by_xpath("//a[@href]")

In [ ]:
print(result[220].text)

In [ ]:
# type(result)

In [ ]:
# print(len(result))

In [ ]:
print(result)

In [22]:
cwd = os.getcwd()

In [23]:
pdf_output_path = Path(f'{cwd}/Data/PDF')

In [24]:
print(pdf_output_path)

C:\Users\chacr\OneDrive\Documents\Coding\Python\USF\COVID-19 Project\Data\PDF


## Downloads the PDF Files from the FDEM Website

In [ ]:
link_counter = 0
len_result = len(result)
for links in result:
    if links.text.find('Data Report') > 0:
        link_url = links.get_attribute('href')
        filename = wget.filename_from_url(link_url)
        if not os.path.exists(pdf_output_path +'/' + filename):
            wget.download(link_url, out = pdf_output_path)
        link_counter +=1
        print (f'{round(link_counter/len_result,3)}% Complete',end='\r')

 

### Attempting to extract PDF text (none found 7-4-20)

In [35]:
print(pdf_file)

<_io.BufferedReader name='C:\\Users\\chacr\\OneDrive\\Documents\\Coding\\Python\\USF\\COVID-19 Project\\Data\\PDF/covid-19-data---daily-report-2020-03-19-0954.pdf'>


In [140]:
county_list = ['Alachua', 'Baker', 'Bay', 'Bradford', 'Brevard', 'Broward', 'Calhoun', 'Charlotte', 'Citrus', 'Clay', 'Collier', 'Columbia', 'Dade', 'Desoto', 'Dixie', 'Duval', 'Escambia', 'Flagler', 'Franklin', 'Gadsden', 'Gilchrist', 'Glades', 'Gulf', 'Hamilton', 'Hardee', 'Hendry', 'Hernando', 'Highlands', 'Hillsborough', 'Holmes', 'Indian River', 'Jackson', 'Jefferson', 'Lafayette', 'Lake', 'Lee', 'Leon', 'Levy', 'Liberty', 'Madison', 'Manatee', 'Marion', 'Martin', 'Monroe', 'Nassau', 'Okaloosa', 'Okeechobee', 'Orange', 'Osceola', 'Palm Beach', 'Pasco', 'Pinellas', 'Polk', 'Putnam', 'Santa Rosa', 'Sarasota', 'Seminole', 'St. Johns', 'St. Lucie', 'Sumter', 'Suwannee', 'Taylor', 'Union', 'Unknown', 'UNKOWN', 'Volusia', 'Wakulla', 'Walton', 'Washington', 
]

In [168]:
file_counter = 0
text_line_search = '^\D+\s\D+\s'#'^[\w+\s\D+].+\Z'
digit_search ='^\d.*\Z'
date_search = '^\w+\s\d.+'
#turn into a function
page_counter = 0


for files in os.listdir(pdf_output_path):
    header = []
    datalines = 0
    print(files)
    pdf_file = open(f'{pdf_output_path}/{files}','rb')
    pdfreader = PdfFileReader(pdf_file)
    num_pages = pdfreader.numPages
    try:
        for pages in range(3,26):
            page =  pdfreader.getPage(pages)   
            if page.extractText().find('Coronavirus: PUI testing by county') > 0 or page.extractText().find('Coronavirus: All persons with tests reported') > 0:
                page_text = page.extractText()
                index_num = 0
                data_row = []
                for lines in page_text.split('\n'):
                    if re.match(date_search, lines, flags=0):
                        date = lines[0:12]
                        print(date)
                    if lines in county_list:
                        datalines = 1
                        data_row = []
                    if not re.match(text_line_search, lines, flags=0) and not re.match(date_search, lines, flags=0) and datalines == 0:
                        header.append(lines)              
                    if datalines == 1:
                        for names in header:
                            if names == 'Awaiting ':
                                header[index_num + 1] = "Awaiting Testing"
                                header.pop(index_num)
                                #print(header)
                            if names == 'Percent ':
                                header[index_num + 1] = 'Pecent Positive'
                                header.pop(index_num) 
                            index_num += 1
                        col_num = len(header)
                        if not re.match(text_line_search, lines, flags=0) and not re.match(date_search, lines, flags=0) and datalines == 1:
                            data_row.append(lines)
        print(header)
    except: IndexError
    file_counter += 1
#     if file_counter == 4:
#         break


covid-19-data---daily-report-2020-03-19-0954.pdf
Mar 19, 2020
Mar 19, 2020
state_reports_20200626.pdf
['County', 'Awaiting Testing', 'Inconclusive', 'Negative', 'Positive', 'Pecent Positive', 'Total tested']
state_reports_20200628.pdf
['County', 'Awaiting Testing', 'Inconclusive', 'Negative', 'Positive', 'Pecent Positive', 'Total tested']
state_reports_20200629.pdf
[]
state_reports_20200630.pdf
['County', 'Awaiting Testing', 'Inconclusive', 'Negative', 'Positive', 'Pecent Positive', 'Total tested']
state_reports_20200701.pdf
['County', 'Awaiting Testing', 'Inconclusive', 'Negative', 'Positive', 'Pecent Positive', 'Total tested']
state_reports_20200702.pdf
['County', 'Awaiting Testing', 'Inconclusive', 'Negative', 'Positive', 'Pecent Positive', 'Total tested']
state_reports_20200703.pdf
['County', 'Awaiting Testing', 'Inconclusive', 'Negative', 'Positive', 'Pecent Positive', 'Total tested']
state_reports_latest_06_27.pdf
['County', 'Awaiting Testing', 'Inconclusive', 'Negative', 'Positi

In [156]:
contents = page.getContents()

In [ ]:
list(contents)

In [ ]:
type(contents[1])

In [25]:
objects = pdfreader.resolvedObjects

NameError: name 'pdfreader' is not defined

In [ ]:
objects

In [ ]:
type(objects)

In [ ]:
objects.keys()

In [ ]:
objects[0, 1]

In [ ]:
objects[0, 1]['/Pages']['/Kids']

In [ ]:
type(objects[0, 1]['/Pages']['/Kids'])

In [ ]:
len(list(umm2[0, 1]['/Pages']['/Kids']))

In [ ]:
umm2[0, 1]['/Pages']['/Kids'].writeToStream()

In [ ]:
import tabula

In [ ]:
df = tabula.read_pdf('data/PDF/AAAAA.pdf' ,pages ='all')

## Extracting a page range from the PDFs

### Steps:
- [X] Open PDFs
- [ ] Extract the page range 12-25
- [ ] Convert that to a word document
- [ ] Find the pages that contain the words "Coronavirus: All persons tested"
- [ ] Extract just those two pages from the Word Document or the 


In [ ]:
print(os.listdir(pdf_output_path))

In [ ]:
pdf_writer = PdfFileWriter()

In [ ]:

def ExtractPDF(input_path):
    import datetime
    today = datetime.date.today()
    pdf_writer = PdfFileWriter()
    
    for files in os.listdir(input_path):
        pdf_file = open(f'{input_path}/{files}','rb')
        print(files)
        print(pdf_file)
        
        print(f'This is the filename: {files}')
        pdfreader = PdfFileReader(pdf_file)
        num_pages = pdfreader.numPages
        for pages in range(12,26):
            page = pdfreader.getPage(pages)
#             print(page)
            pdf_writer.addPage(page)
        output = f'Data/Unclean PDF/Unclean Extract from {files} ({today})'
        with open(output, 'wb') as output_pdf:
            pdf_writer.write(output_pdf)
        break
ExtractPDF(pdf_output_path)

In [ ]:
driver.close()

In [ ]:
os.listdir(cwd)

In [ ]:
print(cwd)
os.chdir('Data/Unclean PDF')
